<a href="https://colab.research.google.com/github/jsebastianwrc/ProyectoFinal_IA_IS2/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos la librerías necesarias para el proyecto

In [16]:
import numpy as np
import pandas as pd 
import matplotlib
from pandas_profiling import ProfileReport

Importamos el archivo CSV base del proyecto

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/resourses/1987 - 2022.csv', header=0)
df = pd.DataFrame(data)

Limpieza de datos

In [ ]:
# TBA meaning (To Be Announced)
data.dtypes

df.drop_duplicates('Title', inplace = True)
df.drop(df[df['Seasons'] == 'TBA'].index, inplace = True)
df.drop(df[df['Episodes'] == 'TBA'].index, inplace = True)
df.drop(df[df['Original Channel'] == ' '].index, inplace = True)

df.loc[df['Final Year'] == 'Present', 'Final Year'] = 2022
df['Final Year'].astype(int)

# Eliminamos las dos primeras columnas
df2 = df.drop(df.columns[[0, 1]],axis = 1)